In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

##########################################################################################################################################################################################################################################

# [1] Functions & Packages for annotating SNPs

##########################################################################################################################################################################################################################################

In [2]:
import vcf

%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from pylab import plot, show, savefig, xlim, figure, hold, ylim, legend, boxplot, setp, axes
from itertools import compress
from pylab import MaxNLocator
import seaborn as sns; sns.set()
from matplotlib.colors import LogNorm
from matplotlib import gridspec
from matplotlib.gridspec import GridSpec
import ast
import itertools
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import fastcluster
from sklearn import cluster, datasets
import scipy.cluster.hierarchy as hier
from sklearn.cluster import KMeans
import time
import sys

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC

from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
import itertools

import networkx as nx
import scipy
import pickle

#for exporting to Adobe Illustrator
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

### Decide on a threshold for Alternate Allele Frequencies to call SNPs

In [3]:
alt_AF_threshold = 0.20 #x% 

### Load regions to exclude from analysis per EBR score across H37Rv (dropping sites with EBR score < 0.9)

In [4]:
with open('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/variant_calls/H37Rv_sites_to_drop.pkl', 'rb') as f:
    H37Rv_positions_to_drop = pickle.load(f)
    
#convert to a set (faster to query)
H37Rv_positions_to_drop = set(H37Rv_positions_to_drop)

## Cell to annotate SNPs

In [5]:
# Important Packages
################################################################################################################################################################################################
import os
import pandas as pd
import numpy as np
import sys
import pickle

import Bio
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
################################################################################################################################################################################################


# Relevant Information for H37Rv sequence SNP functional annotation
################################################################################################################################################################################################
####### Collect all DNA and Amino Acid sequences corresponding to genes on H37Rv #######
#load reference genome and reference annotation
reference_genome = '/n/data1/hms/dbmi/farhat/bin/work-horse/bin/h37rv.fasta'
for reference_genome in SeqIO.parse(reference_genome, "fasta"):
    reference_genome.seq.alphabet = IUPAC.unambiguous_dna

reference_genome_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/H37Rv/h37rv_genome_summary.txt', '\t').set_index('name')

####### Function to translate coding DNA sequences ####### 
def translate(gene_id, sequence):

    #find which strand the gene is located on and translate
    strand = reference_genome_annotation.loc[gene_id, 'strand']
    if strand == '+':
        protein_sequence = sequence.translate(table="Bacterial", cds=False)
    elif strand == '-':
        protein_sequence = sequence.reverse_complement().translate(table="Bacterial", cds=False)

    return protein_sequence

####### Load in dictionaries for SNP annotation #######
with open('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/pickled_files/dicts_for_SNP_annotation/H37Rv_gene_seq_records.pickle', 'rb') as handle:
    ref_gene_sequences_records = pickle.load(handle)
    
with open('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/pickled_files/dicts_for_SNP_annotation/H37Rv_protein_seq_records.pickle', 'rb') as handle:
    ref_protein_sequences_records = pickle.load(handle)
    
with open('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/pickled_files/dicts_for_SNP_annotation/H37Rv_coord_gene_mapping.pickle', 'rb') as handle:
    ReferencePosition_Gene_mapping = pickle.load(handle)
    
####### get Gene Categories #######
gene_categories = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/CSV_files/gene_categories/gene_categories.csv').set_index('name')
gene_categories_dict = dict([gene_id , gene_category] for gene_id, gene_category in zip(list(gene_categories.gene_id) , list(gene_categories.Gene_Category)))

####### get Gene Symbols #######
gene_symbol_dict = dict([gene_id , gene_symbol] for gene_id, gene_symbol in zip(list(reference_genome_annotation.symbol.index) , list( reference_genome_annotation.symbol )))
################################################################################################################################################################################################


# Function to annotate Intergenic SNPs
################################################################################################################################################################################################
def find_flanking_genes_for_intergenic_region(intergenic_ref_pos): 

    #this function finds the genes flagging an intergenic region given a reference position

    #find gene immediately in the 5' direction
    for i in range(0 , 100000):

        #move toward 5' direction
        if ReferencePosition_Gene_mapping[intergenic_ref_pos - i] != []:

            gene_to_left = ReferencePosition_Gene_mapping[intergenic_ref_pos - i][0]
            break

    #find gene immediately in the 3' direction       
    for i in range(0 , 100000):

        #move toward 3' direction
        try:
            if ReferencePosition_Gene_mapping[intergenic_ref_pos + i] != []:

                gene_to_right = ReferencePosition_Gene_mapping[intergenic_ref_pos + i][0]
                break
        
        #KeyError means we have hit the 'end' of the chromosome, the intergenic region at then end of H37Rv in 5' > 3' orientation 
        #since TB chromosome is circular the gene to the 'right' is Rv0001    
        except KeyError:
            
            gene_to_right = 'Rv0001'
            break
    
    return gene_to_left + '_' + gene_to_right
################################################################################################################################################################################################


# Function to determine whether SNPs are Synonymous or Non-Synonymous; Returns gene coordinate, codon position, AA changes, Gene Category & Symbol
################################################################################################################################################################################################
def SNP_annotate(ref_seq_position , alt_allele_i):
    
    '''
    This function takes as input a reference position on H37Rv located within a 
    gene and an alternate allele and returns whether the base change 
    would correspond to a different Amino Acid sequence that results 
    from translating the DNA sequence into an AA sequence.
    
    '''
    gene_intergenic_id_list = []
    genomic_coord_list = []
    gene_category_list = []
    gene_symbol_list = []
    Syn_NSyn_list = []
    AA_change_list = []
    
    #get the Reference Allele from the complete H37Rv reference genome, indexing starts from 0
    ref_allele_i = reference_genome.seq[int(ref_seq_position) - 1] 
    
    #find the gene that SNP occurs on; check list corresponding to H37Rv coordinate to see if there are any genes associated with RefPosition
    if len(ReferencePosition_Gene_mapping[ref_seq_position]) > 0:

        #iterate through all genes that ReferencePosition is mapped to (i.e. SNP might correspond to 2 genes)
        for gene_intergenic_id in ReferencePosition_Gene_mapping[ref_seq_position]:

            #find genomic coordinate of SNP relative to gene (subtract 1 since reference seq starts counting at 1)
            gene_relative_coord = (ref_seq_position - 1) - min( reference_genome_annotation.loc[gene_intergenic_id , 'chromStart'] , reference_genome_annotation.loc[gene_intergenic_id , 'chromEnd'] )
            
            #find the genomic coordinate (relative to the gene, in the 5' to 3' direction)
            strand = reference_genome_annotation.loc[gene_intergenic_id, 'strand']
            if strand == '+':
                 genomic_5_to_3_coord = (ref_seq_position) - reference_genome_annotation.loc[gene_intergenic_id , 'chromStart']

            elif strand == '-':
                 genomic_5_to_3_coord = (reference_genome_annotation.loc[gene_intergenic_id , 'chromEnd']) - (ref_seq_position-1)
                    
            #find gene category (if one exists)
            try:
                gene_category_i = gene_categories_dict[gene_intergenic_id]
            except KeyError:
                gene_category_i = 'None'
            
            #find gene symbol (if one exists)
            try:
                gene_symbol_i = gene_symbol_dict[gene_intergenic_id]
            except KeyError:
                gene_symbol_i = 'None'
            
            #alternate allele is an actual base
            if alt_allele_i in ['A','C','G','T']:

                #translate into protein sequence with the SNP in place if not InDel or intergenic region
                SNP_change = alt_allele_i

                #ALTERNATE allele (is it Syn or NSyn?)
                #get sequence from dictionary of sequences (and convert to mutable object)
                test_gene_sequence = ref_gene_sequences_records[gene_intergenic_id].seq.tomutable()

                #change reference gene sequence by the SNP in the query sequence
                test_gene_sequence[int(gene_relative_coord)] = SNP_change

                #convert back immutable object
                test_gene_sequence = test_gene_sequence.toseq()

                #translate sequence into amino acid seq
                test_protein_sequence = translate(gene_intergenic_id , test_gene_sequence)

                #store the H37Rv AA seq to compare against
                H37Rv_AA_sequence = ref_protein_sequences_records[gene_intergenic_id].seq

                #get the codon number where the SNP occurs within
                ## take the genomic coordinate (relative to the gene, in the 5' to 3' direction), divide by 3, then take the ceiling of this number (will be fraction if SNP occurs in 1st or 2nd position on codon)
                strand = reference_genome_annotation.loc[gene_intergenic_id, 'strand']
                if strand == '+':
                     genomic_5_to_3_coord = (ref_seq_position) - reference_genome_annotation.loc[gene_intergenic_id , 'chromStart']

                elif strand == '-':
                     genomic_5_to_3_coord = (reference_genome_annotation.loc[gene_intergenic_id , 'chromEnd']) - (ref_seq_position-1)

                codon_coord = int(np.ceil( float( genomic_5_to_3_coord) / 3.0 ))

                #compare to AA seq of original gene
                if test_protein_sequence == H37Rv_AA_sequence:

                    SNP_type = 'S'

                    #get the AA before & after
                    AA_change = H37Rv_AA_sequence[codon_coord-1] + str(codon_coord) + test_protein_sequence[codon_coord-1]

                else:
                    SNP_type = 'N'

                    #get the AA before & after
                    AA_change = H37Rv_AA_sequence[codon_coord-1] + str(codon_coord) + test_protein_sequence[codon_coord-1]
                    
            #alternate allele is a dummy (Base Call completely supports the Reference Allele)       
            else:
                
                SNP_type = 'None'
                AA_change = 'None'

            #store relevant info in lists    
            gene_intergenic_id_list.append(gene_intergenic_id)
            genomic_coord_list.append(genomic_5_to_3_coord)
            gene_category_list.append(gene_category_i)
            gene_symbol_list.append(gene_symbol_i)
            Syn_NSyn_list.append(SNP_type)
            AA_change_list.append(AA_change)
    
    #if no gene in H37Rv corresponds to the Reference Position for SNP, then SNP must be intergenic
    else:
        
        gene_intergenic_id = find_flanking_genes_for_intergenic_region(ref_seq_position)
        genomic_5_to_3_coord = 'None'
        gene_category_i = 'None'
        gene_symbol_i = 'None'
        SNP_type = 'I'
        AA_change = 'None'
        
        #store relevant info in lists    
        gene_intergenic_id_list.append(gene_intergenic_id)
        genomic_coord_list.append(genomic_5_to_3_coord)
        gene_category_list.append(gene_category_i)
        gene_symbol_list.append(gene_symbol_i)
        Syn_NSyn_list.append(SNP_type)
        AA_change_list.append(AA_change)
    
    #if there is only a single gene associated with this SNP, just return the individual elememts
    if len(gene_intergenic_id_list) == 1:
        return [ref_allele_i , gene_intergenic_id , genomic_5_to_3_coord , gene_category_i , gene_symbol_i , SNP_type , AA_change]
    
    #else if there are two genes associated with this SNP, return elements for each SNP annotation in a list
    elif len(gene_intergenic_id_list) > 1:
        return [ref_allele_i , gene_intergenic_id_list , genomic_coord_list , gene_category_list , gene_symbol_list , Syn_NSyn_list , AA_change_list]
################################################################################################################################################################################################

## Function to get parse through and annotate SNPs (filtered for $AF$, MGE and low EBR score regions)

In [6]:
def annotate_and_filter_SNPs(sample_tag , alt_AF_threshold):

    '''
    This function annotates SNPs (Base Calls) extracted from VCF files. 
    This function drops SNPs called with an alternate allele frequency (AF) less than x%.
    This function also drops SNPs in regions flagged as Mobile Genetic Elements & Regions of poor Illumina mapping / variant calling
    per Empirical Base Recall (EBR) scores across H37Rv.
    '''

    ################################################################################
    ### get SNPs that were extracted from VCF files
    ################################################################################

    #load in the differing Base Calls for the isolate pair from pickle
    SNPs_from_VCF = pd.read_pickle(parent_output_dir + sample_tag +  '/' + sample_tag + '_SNP_calls.pkl')

    ################################################################################
    ### Drop SNPs with change in AF < x%
    ################################################################################

    #FILTER out paired Base Calls that have an in Alternate Allele Frequency of less than x%
    SNPs_from_VCF = SNPs_from_VCF[SNPs_from_VCF.alt_AF >= alt_AF_threshold]

    #reset index of filtered SNP DataFrame
    SNPs_from_VCF.reset_index(inplace = True, drop = True)

    ################################################################################
    ### Drop SNPs in regions with low EBR scores
    ################################################################################

    #Drop Base Calls in H37Rv sites with low EBR score (make sure there is at least 1 SNP)
    if np.shape(SNPs_from_VCF)[0] > 0:

        #create a boolean filter for SNPs to keep
        SNPs_to_keep_filter = [SNP_i_ref_pos not in H37Rv_positions_to_drop for SNP_i_ref_pos in SNPs_from_VCF.ref_position]

        #filter out SNPs in H37Rv sites with low EBR scores and reset index
        SNPs_from_VCF = SNPs_from_VCF[SNPs_to_keep_filter]
        SNPs_from_VCF.reset_index(inplace = True, drop = True)

    ################################################################################
    ### Annotate SNPs & Drop SNPs in MGE regions
    ################################################################################

    gene_id_list = []
    gene_coord_list = []
    gene_category_list = []
    gene_symbol_list = []
    SNP_ftype_list = []
    AA_change_list = []

    #Annotate Filtered Base Calls (make sure there is at least 1 SNP)
    if np.shape(SNPs_from_VCF)[0] > 0:

        for ref_position_i , alt_base_i in zip(list(SNPs_from_VCF.ref_position) , list(SNPs_from_VCF.alt_base)):

            #annotate SNP
            gene_id_i , gene_coord_i , gene_category_i , gene_symbol_i , SNP_ftype_i , AA_change_i = SNP_annotate(ref_position_i , alt_base_i)[1:]

            gene_id_list.append(gene_id_i)
            gene_coord_list.append(gene_coord_i)
            gene_category_list.append(gene_category_i)
            gene_symbol_list.append(gene_symbol_i)
            SNP_ftype_list.append(SNP_ftype_i)
            AA_change_list.append(AA_change_i)

        #create columns to store SNP annotation info
        SNPs_from_VCF['gene_id'] = gene_id_list
        SNPs_from_VCF['gene_coord'] = gene_coord_list
        SNPs_from_VCF['gene_category'] = gene_category_list
        SNPs_from_VCF['gene_symbol'] = gene_symbol_list
        SNPs_from_VCF['SNP_ftype'] = SNP_ftype_list
        SNPs_from_VCF['AA_change'] = AA_change_list

        #FILTER out Base Calls in MGE regions (Mobile Genentic Elements)
        SNPs_to_drop_filter = [] #True if SNP is located within an MGE region

        for gene_id_i in list(SNPs_from_VCF.gene_category):

            #only 1 or 0 genes associated with this SNP
            if (type(gene_id_i) == str) and (gene_id_i == 'Mobile Genetic Element'):

                SNPs_to_drop_filter.append(True)

            #two genes associated with this SNP
            elif (type(gene_id_i) == list) and ('Mobile Genetic Element' in gene_id_i):

                SNPs_to_drop_filter.append(True)

            #SNP not in an MGE region so don't drop
            else:

                SNPs_to_drop_filter.append(False)

        #create a boolean filter for SNPs to keep
        SNPs_to_keep_filter = [not MGE_SNP for MGE_SNP in SNPs_to_drop_filter]

        #filter out SNPs in MGE regions and reset index
        SNPs_from_VCF = SNPs_from_VCF[SNPs_to_keep_filter]
        SNPs_from_VCF.reset_index(inplace = True, drop = True)

    #No SNPs detected between this pair of isolates (empty DataFrame)
    else:

        SNPs_from_VCF['gene_id'] = ""
        SNPs_from_VCF['gene_coord'] = ""
        SNPs_from_VCF['gene_category'] = ""
        SNPs_from_VCF['gene_symbol'] = ""
        SNPs_from_VCF['SNP_ftype'] = ""
        SNPs_from_VCF['AA_change'] = ""

    #drop unnecessary columns
    SNPs_from_VCF.drop(['quality','SNP_type','PASS_filter','INFO'] , axis = 1 , inplace = True)

    return SNPs_from_VCF

##########################################################################################################################################################################################################################################

# [2] Use functions above to annotate & filter SNPs

##########################################################################################################################################################################################################################################

#### Import Sample Annotation file

In [7]:
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/annotation CSV files/sample_annotation.csv' , sep  = ',').set_index('tag')

#### output where *pilon*-output is stored

In [8]:
parent_output_dir = '/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/JankyPipe/output/'

In [9]:
sample_annotation.head()

,sample_id,sample_num,fastq_files,coverage,lineage_call
tag,,,,,
S0,Undetermined,0,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,25.5521,4.6.1.2
S1,TC100810,1,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,151.0760,4.9
S2,TC94815,2,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,60.5541,4.3.4.2.1
S3,TC101527,3,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,70.5301,3
S4,LB95669,4,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,82.3331,2.2.1


In [10]:
np.shape(sample_annotation)

(24, 5)

### Collect SNPs passing Difference in Alternate Allele Frequency Threshold

In [11]:
SNPs_from_samples_df_list = []

sample_index = 0

#iterate through isolate pairs, collect all SNP variants called in each sample
for sample_tag in sample_annotation.index:
    
    #retrieve filtered & annotated SNPs
    SNP_variants_sample_i = annotate_and_filter_SNPs(sample_tag , alt_AF_threshold)
    
    #store relevant SNP info in list of DataFrames
    SNPs_from_samples_df_list.append(SNP_variants_sample_i)
    sample_index += 1
    
    print sample_index
        
#concatenate DataFrames for each sample into 1 DataFrame
SNPs_from_all_samples_df = pd.concat(SNPs_from_samples_df_list , axis = 0)

SNPs_from_all_samples_df.reset_index(inplace = True , drop = True)

/home/rv76/anaconda2/lib/python2.7/site-packages/Bio/Seq.py:2715: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [12]:
SNPs_from_all_samples_df.head()

,ref_base,alt_base,ref_position,alt_AF,depth,tag,gene_id,gene_coord,gene_category,gene_symbol,SNP_ftype,AA_change
0,C,A,1849,0.38,59,S0,Rv0001_Rv0002,None,None,None,I,None
1,A,G,1977,0.90,50,S0,Rv0001_Rv0002,None,None,None,I,None
2,G,C,7362,0.93,26,S0,Rv0006,61,Antibiotic Resistance,gyrA,N,E21Q
3,G,C,7585,0.88,19,S0,Rv0006,284,Antibiotic Resistance,gyrA,N,S95T
4,T,C,9143,0.25,24,S0,Rv0006,1842,Antibiotic Resistance,gyrA,S,I614I


In [13]:
np.shape(SNPs_from_all_samples_df)

(24782, 12)

#### Save output as a CSV files and pickled DataFrame

In [14]:
SNPs_from_all_samples_df.to_csv('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/variant_calls/SNPs/SNP_calls_across_all_samples.csv')
SNPs_from_all_samples_df.to_pickle('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/variant_calls/SNPs/SNP_calls_across_all_samples.pkl')